In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress

# Import API key
weather_api_key = "(input your key here)"

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

## Perform API Calls
    * Perform a weather check on each city using a series of successive API calls.
    * Include a print log of each city as it's being processed (with the city number and city name)

In [ ]:
# Starting URL for Weather Map API Call
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
query_url = f"{url}appid={weather_api_key}&units={units}&q="

print(requests.get(query_url + city))

In [ ]:
# Set empty lists to hold city data
city_name = []
lat = []
lng = []
max_temp = []
humidity = []
cloudiness = []
wind_speed = []
country = []
date = []

# Set record & set at the start
record = 1

# Print starting statement
print(f"Beginning Data Retrieval")
print(f"-------------------------------")

# Loop through each city 
for city in cities:
    
    # Make request and store in json
    response = requests.get(query_url + city).json()
    
    # Try to append city info
    try:
        city_name.append(response["name"])
        lat.append(response["coord"]["lat"])
        lng.append(response["coord"]["lon"])
        max_temp.append(response["main"]["temp_max"])
        humidity.append(response["main"]["humidity"])
        cloudiness.append(response["clouds"]["all"])
        wind_speed.append(response["wind"]["speed"])
        country.append(response["sys"]["country"])
        date.append(response["dt"])
        city_record = response["name"]
        record += 1
        
        # Print statement
        print(f"Processing Record {record} | {city_record}")
        
        # Slow loop in order not to exceed rate limit of API
        time.sleep(1.01)
        
    # Handle exceptions for a city that is not available
    except:
        print("City not found. Go to the next one.")
        pass

# Print Ending Log Statement
print(f"-------------------------------")
print(f"Data Retrieval Complete")
print(f"-------------------------------")

## Convert Raw Data to DataFrame
    * Export the city data into a .csv.
    * Display the DataFrame

In [ ]:
# Create a DataFrame from Cities, Latitude, Longitude, Max Temperature, Humidity, Cloudiness, Wind Speed, Country, Date
weather_info = {
    "City": city_name,
    "Lat": lat, 
    "Lng": lng, 
    "Max Temp": max_temp,
    "Humidity": humidity,
    "Cloudiness": cloudiness,
    "Wind Speed": wind_speed,
    "Country":country,
    "Date": date,   
}

weather_df = pd.DataFrame(weather_info)

# Display DataFrame 
weather_df.head()

In [ ]:
# Display a statistical overview of the DataFrame
weather_df.describe()

In [ ]:
# Save DataFrame to CSV
weather_df.to_csv("output_data/cities.csv", index = False)

## Inspect the data and remove the cities where the humidity > 100%
Skip this step if there are no cities that have humidity > 100%.

In [ ]:
# Get the indices of cities that have humidity over 100%.
test_stats = weather_df["Humidity"].describe(include = "all")
test_stats

Answer: max humidity is 100%, so no outliers

## Plotting the Data

Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
Save the plotted figures as .pngs.

### Latitude vs. Temperature Plot

In [ ]:
# Date of analysis
today = pd.to_datetime('today').normalize() 

# Build a scatter plot for each data type
plt.scatter(weather_df["Lat"], weather_df["Max Temp"], marker="o", s=30, edgecolor="black")

# Incorporate the other graph properties
plt.title("City Latitude vs. Maximum Temperature (today)")
plt.ylabel("Maximum Temperature (F)")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure
plt.savefig("Images/City Latitude vs Max Temperature.png")

# Show plot
plt.show()

### Latitude vs. Humidity Plot

In [ ]:
# Build a scatter plot for each data type
plt.scatter(weather_df["Lat"], weather_df["Humidity"], marker="o", s=30, edgecolor="black")

# Incorporate the other graph properties
plt.title("City Latitude vs. Humidity (today)")
plt.ylabel("Humidity (%)")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure
plt.savefig("Images/Humidity_vs_Latitude.png")

# Show plot
plt.show()

###  Latitude vs. Cloudiness Plot

In [ ]:
# Build a scatter plot for each data type
plt.scatter(weather_df["Lat"], weather_df["Cloudiness"], marker="o", s=30, edgecolor="black")

# Incorporate the other graph properties
plt.title("City Latitude vs. Cloudiness (today)")
plt.ylabel("Cloudiness (%)")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure
plt.savefig("Images/Cloudiness_vs_Latitude.png")

# Show plot
plt.show()

###  Latitude vs. Wind Speed Plot

In [ ]:
# Build a scatter plot for each data type
plt.scatter(weather_df["Lat"], weather_df["Wind Speed"], marker="o", s=30, edgecolor="black")

# Incorporate the other graph properties
plt.title("City Latitude vs. Wind Speed (today)")
plt.ylabel("Wind Speed (mph)")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure
plt.savefig("Images/Wind_Speed_vs_Latitude.png")

# Show plot
plt.show()

###  Linear Regression

In [ ]:
northern = weather_df.loc[weather_df["Lat"] >= 0.0]
northern.reset_index(inplace=True)

southern = weather_df.loc[weather_df["Lat"] < 0.0]
southern.reset_index(inplace=True)

    Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
## Convert to floats and store weather values as x and y values
x_values = northern['Lat'].astype('float')
y_values = northern['Max Temp'].astype('float')

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.xlabel('Lat')
plt.ylabel('Max Temp (F)')

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()

    Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
## Convert to floats and store weather values as x and y values
x_values = southern['Lat'].astype('float')
y_values = southern['Max Temp'].astype('float')

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.xlabel('Lat')
plt.ylabel('Max Temp (F)')

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()

     Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
## Convert to floats and store weather values as x and y values
x_values = northern['Lat'].astype('float')
y_values = northern['Humidity'].astype('float')

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.xlabel('Lat')
plt.ylabel('Humidity (%)')

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()

     Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
## Convert to floats and store weather values as x and y values
x_values = southern['Lat'].astype('float')
y_values = southern['Humidity'].astype('float')

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.xlabel('Lat')
plt.ylabel('Humidity (%)')

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()

     Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
## Convert to floats and store weather values as x and y values
x_values = northern['Lat'].astype('float')
y_values = northern['Cloudiness'].astype('float')

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.xlabel('Lat')
plt.ylabel('Cloudiness (%)')

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()

    Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
## Convert to floats and store weather values as x and y values
x_values = southern['Lat'].astype('float')
y_values = southern['Cloudiness'].astype('float')

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.xlabel('Lat')
plt.ylabel('Cloudiness (%)')

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()

    Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
## Convert to floats and store weather values as x and y values
x_values = northern['Lat'].astype('float')
y_values = northern['Wind Speed'].astype('float')

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.xlabel('Lat')
plt.ylabel('Wind Speed (mph)')

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()

    Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
## Convert to floats and store weather values as x and y values
x_values = southern['Lat'].astype('float')
y_values = southern['Wind Speed'].astype('float')

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.xlabel('Lat')
plt.ylabel('Wind Speed (mph)')

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()